## Building A Chatbot
We will design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This is a basic chatbot
1. Load the environment variables from the .env file.
2. Store these env variable in a separate variable used further to access the models.
3. Select and load your model. In our case, we are using a Gemma model from chatgroq, langchain.
4. Chatting with chatbot to see if it remembers previous history.

##### In case of multiple users, we need to maintain session and session data for each chat.
5. We will use RunnableWithMessageHistory object.

##### 

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()   # loading all the environment variable


True

In [4]:
# Groq - AI inference for faster compute and memory
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x114ee8050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x114ee8d70>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
# Start with basic text
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Divya Gupta and I am a Lead Developer")])

# Here I have not passed anything from the SystemMessage. 
# SystemMessage will tell what kind of behaviour the LLM will perform.
# AIMessage : is the response from LLM Model.

AIMessage(content="Hi Divya Gupta, it's nice to meet you!  \n\nBeing a Lead Developer is a great role. What kind of projects do you typically work on?  \n\nI'm happy to chat about anything you'd like, whether it's about your work, technology, or just something else entirely. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 23, 'total_tokens': 92, 'completion_time': 0.125454545, 'prompt_time': 0.002121366, 'queue_time': 0.160019053, 'total_time': 0.127575911}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--da05660c-bfd7-4386-9c05-59d5b27d406b-0', usage_metadata={'input_tokens': 23, 'output_tokens': 69, 'total_tokens': 92})

In [ ]:
# We are giving a list of messages. And giving some previous information.
# The chatbot is remembering the previous information too.

from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Divya Gupta and I am a Lead Developer"),
        AIMessage(content="Hi Divya Gupta, it's nice to meet you!  \n\nBeing a Lead Developer is a great role. What kind of projects do you typically work on?  \n\nI'm happy to chat about anything you'd like, whether it's about your work, technology, or just something else entirely. 😊\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Divya Gupta, and you are a Lead Developer!  \n\nIs there anything else you'd like to know or discuss? 😊  I'm here to help. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 112, 'total_tokens': 153, 'completion_time': 0.074545455, 'prompt_time': 0.007408926, 'queue_time': 0.161532398, 'total_time': 0.081954381}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a9ee539e-db64-40aa-9f18-a78c3c3df157-0', usage_metadata={'input_tokens': 112, 'output_tokens': 41, 'total_tokens': 153})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory #every next message should be the part of the previous conversation.

store={}

# Distinguish one chat session with the other
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory() # whatever chat is happening will go to this session id.
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

# Here : 
# 1. we create a dictonary.
# 2. Then we created a function, which will retrieve, whatever chat history is there for a specific session id.
# 3. Then we are converting/using the RunnableWithMessageHistory, so we can interact with our LM model based on the chat history.


In [11]:
config={"configurable":{"session_id":"chat1"}}

In [14]:
response = with_message_history.invoke([HumanMessage(content="Hi , My name is Divya Gupta and I am a Lead Developer")],
    config=config)

In [15]:
response.content

"Hi Divya Gupta!  \n\nIt's great to meet you.  Being a Lead Developer is a challenging and rewarding role. What kind of projects are you currently leading?  \n\nI'm curious to hear about the technologies you're using and the problems you're solving.\n"

In [16]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Divya Gupta.  \n\nI remember from our earlier conversation. 😊  How can I help you today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 171, 'total_tokens': 200, 'completion_time': 0.052727273, 'prompt_time': 0.008641406, 'queue_time': 0.16259483400000002, 'total_time': 0.061368679}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0b593633-8165-4cfb-8832-ec69b7f64de8-0', usage_metadata={'input_tokens': 171, 'output_tokens': 29, 'total_tokens': 200})

In [17]:
# Till now you are part of seesion id 1. 
# Now lets change the session id and see if your chatbot will be able to remember your conversation

## change the config-->session id
config2={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config2
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [18]:
# Feeding information to the session2.
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config2
)
response.content

"Hi John! It's nice to meet you. 😊 \n\nWhat can I do for you today?\n"

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config2
)
response.content

"Your name is John, that's what you told me! 😊  \n\nIs there anything else I can help you with?\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [ ]:
# Whatever human messages we give, it should be in the key:value pair. Here key is messages.
# Above we have given messages like HumanMessage, AIMessage, SystemMessage. SO, they are are the keys.
# But now the key will be messgaes

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [21]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Divya")]})

AIMessage(content="Hi Divya! It's nice to meet you. \n\nI'm happy to answer any questions you have to the best of my ability. Just ask away! 😊  What can I help you with today? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 32, 'total_tokens': 81, 'completion_time': 0.089090909, 'prompt_time': 0.002353145, 'queue_time': 0.161425205, 'total_time': 0.091444054}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--73bc8d5d-b827-438e-bbe3-7f9ec9683435-0', usage_metadata={'input_tokens': 32, 'output_tokens': 49, 'total_tokens': 81})

In [22]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [25]:
config3 = {"configurable": {"session_id": "chat3"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Jenny")],
    config=config3
)

response

AIMessage(content="Hi Jenny! It's nice to meet you. 😊 \n\nI'm ready to answer your questions. What can I do for you today?  \n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 132, 'total_tokens': 168, 'completion_time': 0.065454545, 'prompt_time': 0.008660315, 'queue_time': 0.160403096, 'total_time': 0.07411486}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--eb0637f4-ff96-4893-a569-1cdaa2828389-0', usage_metadata={'input_tokens': 132, 'output_tokens': 36, 'total_tokens': 168})

In [26]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config3,
)

response.content

'Your name is Jenny!  \n\nHow can I help you further? 😄  \n'

In [27]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [28]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Divya")],"language":"Hindi"})
response.content

'नमस्ते, दीव्या!  😊\n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। आप मुझसे जो भी प्रश्न पूछना चाहें, मैं अपनी पूरी कोशिश करूँगा कि आपको सबसे अच्छे जवाब दूँ।  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [31]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [32]:
config = {"configurable": {"session_id": "chat4"}}

repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Divya")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Divya! 😊  \n\nमुझे आपसे मिलकर अच्छा लगा!  आप कैसे हैं?  \n'

In [33]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [34]:
response.content

'आपका नाम Divya है।  😌 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.


'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [39]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2 + 2 "),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2 + 2 ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
# I HAVE did trimmer. Now I am asking questions based on the conversations I had earlier. 
# Since the question asked is trimmed and is not available in the ............., we got no response.


from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences.  What's your favorite flavor? 😊🍦\n"

In [41]:
# Asking some recent question, which is not being trimmed.

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 equals. 😊  \n\n\n\nLet me know if you want to try another one!\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nWould you like to tell me your name? 😊  \n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. So I don't know what math problem you asked. \n\nIf you'd like to ask me a math problem now, I'm happy to help! 😊  \n\n"